In [5]:
import torch
import torch.nn as nn
import torch.onnx
import onnx
import pydot
from onnx.tools.net_drawer import GetPydotGraph


# 定义一个简单的 PyTorch 模型
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.fc1 = nn.Linear(3, 5)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(5, 2)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x


# 初始化模型并设置为评估模式
model = SimpleModel()
model.eval()

# 创建一个示例输入
dummy_input = torch.randn(1, 3)  # 批大小为1，输入特征维度为3

# 导出为 ONNX 格式
onnx_file_path = "simple_model.onnx"
torch.onnx.export(
    model,
    dummy_input,
    onnx_file_path,
    export_params=True,  # 保存权重
    opset_version=11,   # 使用的 ONNX opset 版本
    input_names=["input"],  # 输入节点的名称
    output_names=["output"],  # 输出节点的名称
    dynamic_axes={"input": {0: "batch_size"}, "output": {0: "batch_size"}},  # 支持动态批大小
)

print(f"Model has been exported to {onnx_file_path}")

# 加载 ONNX 模型
onnx_model = onnx.load(onnx_file_path)


# 新增：将 ONNX 模型可视化为图像
def visualize_onnx(onnx_model_path, output_image_path):
    model = onnx.load(onnx_model_path)
    # 生成网络图
    pydot_graph = GetPydotGraph(
        model.graph,  # 注意参数传递的是 `models.graph`
        name=model.graph.name,
        rankdir="TB",  # 图形方向："TB" 表示从上到下
    )
    # 保存为 PNG 图像
    with open(output_image_path, "wb") as f:
        f.write(pydot_graph.create_png())
    print(f"ONNX model visualization saved as image at {output_image_path}")


# 调用可视化函数
visualize_onnx(onnx_file_path, "simple_model_visualization.png")

Model has been exported to simple_model.onnx
ONNX model visualization saved as image at simple_model_visualization.png
